# Workflow for genomic data visualization using PyGenomeTrack
# Part 1: Data Preparation

***
>This notebook contains a procedure to prepare the sequencing data and to create the region plots. 
The workflow describes the tools to convert the files into the formats: BED6, BED12, BAM, and BIGWIG. Additionally, we formed the helper functions to create .ini files from scratch.

> The [PyGenomeTrack](https://github.com/deeptools/pyGenomeTracks) aims to produce high-quality genome browser tracks.

> The main steps:
    0. Installation of requirements.
    1. Prepare your .gff3 files.
    2. Convert .gff3 into .bed6.
    3. Convert .gff3 into .bed12.
    4. Make .bigwig file from BAM/SAM format.
    5. Prepare the .ini files - from scrach or by edition of the example file.

> __HINT:__ Exclamation mark (!) at the beginning of line allows to use bash commands from the jupyter notebook.
***

***
### Types of file formats used in this Notebook:
* [GFF3](https://www.ensembl.org/info/website/upload/gff3.html) - General Feature Format Version 3
* [BED6/BED12](https://genome.ucsc.edu/FAQ/FAQformat.html#format1) - Browser Extensible Data
* The [BIGWIG](https://genome.ucsc.edu/goldenPath/help/bigWig.html) format is useful for dense, continuous data that will be displayed in the Genome Browser as a graph.
* The __INI__ file format is an informal standard for configuration files.
* [SAM](https://samtools.github.io/hts-specs/SAMv1.pdf) - Sequence Alignment Map
* [BAM](https://genome.ucsc.edu/goldenPath/help/bam.html) is the compressed binary version of the Sequence Alignment/Map (SAM) format, a compact and index-able representation of nucleotide sequence alignment.
* [genePred](http://genome.ucsc.edu/FAQ/FAQformat#format9) is a table format commonly used for gene prediction tracks in the Genome Browser.
***

## 0. Installation of requirements

### A) Installation from file 'requirements.txt' using "pip install"
> __HINT:__ We recommend to install the requirements in a virtual environment to avoid inconsistencies in the local environment.

In [5]:
!pip install -r requirements.txt 

### B)   Following tools couldn't be installed with "pip install". Install it in a different way, e.g., with bioconda.

### Please open the terminal and install following tools step by step: 

* Installation of gff3togenepred

__conda install ucsc-gff3togenepred__

* Update of gff3togenepred

__conda update ucsc-gff3togenepred__

* Installation of GenePredToBed

__conda install -c bioconda ucsc-genepredtobed__

* Installation of bedtools - example

__sudo apt install bedtools__

__sudo apt install bedops__

__sudo apt install samtools__



### C) Besides, download [BEDOPS](https://bedops.readthedocs.io/en/latest/) for your operating system and install it. BEDOPS contains samtools, sortbed, and bamtools.


***
## 1. Prepare your GFF3 and SAM/BAM files

> Please look carefully at your GFF3 file. Delete incorrect lines in your file.
***

***
## 2. Convert .gff3 into .bed6
    sortBed -i <gff3_file.gff3> | gff2bed > <bed_file.bed>
***

In [9]:
# Example
!sortBed -i newSP.gff3 | gff2bed > file_bed6.bed

In [11]:
# Sort Bed6:
!sort -k 1,1 -k2,2n file_bed6.bed > file_bed6_sorted.bed

In [13]:
# Sort Bed12:
!sort -k 1,1 -k2,2n file_bed12.bed > file_bed12_sorted.bed

***
## 3. Convert .gff3 into .bed12
    gff3ToGenePred infile.gff3 temp.genePre
    
> __HINT:__ Possible error 'CDS feature must have phase'. 
Phase is a parameter in column 8 of gff3 (only for features CDS).
Add phases in the gff3 file according to the notes below.
You can open your GFF3 file and add 0 in column 8 in the lines with CDS, e.g. using Regular Expressions
In case of another type of error, e.g. "Error: invalid genePred created: SPAC17G8.01c.1 I:2341250-2343947", look carefully at your gff3 and check if your exons are overlapping.


> "Phase specifies how many nucleotides must be removed from the beginning of a CDS to reach a complete codon 
(0, 1 or 2). If phase is missing, the CDS cannot be translated without making dangerous assumptions about 
the data. Specifically, you have to assume that the first CDS has a phase of 0. For poorly assembled genomes, 
this may not by the case.
Even if the assumption always holds, a CDS without phase data can be hard to work with since you cannot 
translate an individual CDS without considering the other members of the model.
According to the GFF specification, phase information is required for CDS entries and is stored 
in the 8th column." (Source: https://support.bioconductor.org/p/101245/ available: 2019.04.03)
***

In [17]:
# Sort the GFF3 file by column 1 (chromosomes) and 4 (start positions) as:
!sort -k1,1 -k4,4n newSP_phase_sort.gff3 > newSP_phase_sorted.gff3

In [18]:
# Creating the temporary file .genePred from gff3
!gff3ToGenePred newSP_phase_sorted.gff3 temp.genePred 

#"newSP_phase.gff3" is a gff3 file with phases in column 8.

In [19]:
# Creating BED file from temporary file
!genePredToBed temp.genePred file_bed12.bed

***
## 4. Make .bigwig file

### A) Conversion of SAM format to BAM format (if your data are in SAM)
        samtools view -S <sam_file.sam> -b -o <bam_file.bam>

In [7]:
# Convert SAM to BAM
# SAM file is too heavy with upload it on Github. 
# Below there is an example of commend, unfortunately without the file "C1_long.sam"

!samtools view -S C1_long.sam -b -o C1_long.bam

[E::hts_open_format] Failed to open file C1_long.sam
samtools view: failed to open "C1_long.sam" for reading: No such file or directory


### B) Conversion of BAM format to BIGWIG using [bamCoverage](https://deeptools.readthedocs.io/en/develop/content/tools/bamCoverage.html#Required%20arguments) from deeptools

> Steps:
    1. Sort BAM file
        samtools sort -n <bam_file.bam>
    2. Add index to bam file
        samtools index <bam_file.bam>
    3. Convert indexed bam to bigwig
        bamCoverage -b <bamfile.bam> -o <bibwigfile.bw>
***

### Sort and add index to BAM file - it is a necessary step

#### __HINT:__ It works better in terminal.

In [ ]:
# Sort the BAM file 
!samtools sort -n C1_long.bam

In [2]:
# Add index to BAM file
!samtools index -b C1_long.bam

^C


### Make BIGWIG from indexed BAM

Run the code in the terminal:

In [3]:
# Convert indexed bam to bigwig
!bamCoverage -b C1_long.bam -o bigwig.bw

'C1_long.bam' does not appear to have an index. You MUST index the file first!


### [Separate tracks](https://deeptools.readthedocs.io/en/develop/content/tools/bamCoverage.html) for each strand (forward and reverse strands):

In [ ]:
# BigWig forward
!bamCoverage -b C1_long.bam -o bigwig_forward.bw --filterRNAstrand reverse

In [ ]:
# BigWig reverse
!bamCoverage -b C1_long.bam -o bigwig_reverse.bw --filterRNAstrand forward

### Short look at the bigwig file

In [4]:
import pyBigWig

bw = pyBigWig.open("bigwig.bw")
bw.header()

{'version': 4,
 'nLevels': 3,
 'nBasesCovered': 12631379,
 'minVal': 0,
 'maxVal': 952468,
 'sumData': 6263806273,
 'sumSquared': 509597617963475}

***
## 5. Preparing the INI file

### A) Preparing the INI file by editing an existing file
>You can prepare your INI file from scratch (protocol below). However, we strongly recommend to use previously prepared INI files, and edit them (using helper functions or in a text editor).

>The examples of color names are [here](https://www.w3schools.com/colors/colors_picker.asp).

### B) Preparing of INI file from scratch -- using helper functions in ini_edit.py
> The helper functions allow to prepare a new ini file or to add some dictionary, section or file to existing ini file.

> All prepared helper functions are saved in file __ini_edit.py__. You can import the content of the file as below, and use all of the function in Python.

> __List of helper functions in ini_edit.py__: 
    * czytaj_konfig(nazwa_pliku)
    * zapisz_ini_ze_slownik(slownik, nazwa_pliku)
    * add_ini_slownik(slownik, nazwa_pliku, nowa_nazwa=None)
    * dopisz_do_konfig(konfig, slownik)

>#### Use belowed commend to look at the description of one of the helper functions in ini_edit.py :
    ?? ini_edit.add_ini_slownik  
***

In [5]:
# Import ini_edit.py where are all helper functions
import ini_edit

In [6]:
# To look at the functions in 
?? ini_edit.add_ini_slownik

### The helper functions are also here: (PS. Some names are in Polish - I will change it soon)

In [11]:
"""
the 'helper functions' for reading and editing the INI files

słownik do ini ma zawsze formę:

slownik[nazwa_sekcji][nazwa_elementu]= wartosc_elementu

i to się zamieni w pliku ini na:

[nazwa sekcji]
nazwa_elementu = wartosc_elementu

"""
import configparser


def czytaj_konfig(nazwa_pliku):
    pass


def zapisz_ini_ze_slownik(slownik, nazwa_pliku):
    """zapisuje konfigurację ini na podstawie słownika

    Args:
        slownik (dict): slownik do zapisania
        nazwa_pliku (str): nazwa/ sciezka do pliku

    Slownik musi mieć formę, że dla każdego klucza,
    oznaczającego sekcję
    jest kolejny słownik zawierający poszczególne
    elementy
    
    Examples:
        >>> slownik={"UW":{"Faculty_of_Biology":"IGIB"}}
        >>> slownik["x-axis"]={"fontsize":20}
        >>> slownik["x-axis"]["where"]="top"
        >>> zapisz_konfig(slownik,"nowa_li_slownik.ini")
        plik nowa_li_slownik.ini zapisany

    """
    config = configparser.RawConfigParser()
    config.read_dict(slownik)
    config.write(open(nazwa_pliku, "w"))
    print("plik {} zapisany".format(nazwa_pliku))


def add_ini_slownik(slownik, nazwa_pliku, nowa_nazwa=None):
    """ Funkcja do pliku ini dopisuje dane ze słownika
    
    Args:
        slownik (dict): slownik z sekcjami i elementami
        nazwa_pliku (path): ścieżka do pliku ini który jest bazą do zmian
        nowa_nazwa (path): ścieżka do pliku ini który będzie wynikowym plikiem, plik opcjonalny (możemy nadpisać nazwa_pliku)
    
    Examples:
        >>> slownik_z_nowymi_rzeczami={"UW":{"Faculty_of_Biology":"IGIB"}}
        >>> add_ini_slownik(slownik_z_nowymi_rzeczami,"testtt.ini","nowy_nowy.ini")
    """
    config = configparser.RawConfigParser()
    config.read(nazwa_pliku)
    dopisz_do_konfig(config, slownik)
    if nowa_nazwa:
        nazwa_pliku=nowa_nazwa
    
    config.write(open(nazwa_pliku,"w"))
    print("zapisano pod nazwą", nazwa_pliku)


def dopisz_do_konfig(konfig, slownik):
    """do obiektu konfig dopisuje (w miejscu) elementy ze slownika
    Args:
        konfig (configparser.RawConfigParser): obiekt zawierajacy konfigurację
        slownik (dict): slownik z sekcjami i elementami
        
    """ 
    
    for nazwa_sekcji,slownik_sekcji in slownik.items():
        
        try:
            konfig.add_section(nazwa_sekcji)
        except :
            print("sekcja {} już istniała".format(nazwa_sekcji))
        
        for nazwa_el,wartosc_el in slownik_sekcji.items():
            konfig.set(nazwa_sekcji,nazwa_el,wartosc_el)
    

***
#### ~ Open PGT_DEMO_Part2_Genome 20Visualization.ipynb to visualize the genome~
***